**Training dan Visualisasi Decision Tree**

Untuk melatih Decision Tree, digunakan kelas `DecisionTreeClassifier` dari Scikit-Learn. Contoh di bawah melatih Decision Tree pada dataset Iris:

```python
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:]  # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)
```

Kode ini memuat dataset Iris, memilih fitur panjang dan lebar kelopak, kemudian melatih `DecisionTreeClassifier` dengan `max_depth` 2.

Visualisasi Decision Tree dapat dilakukan menggunakan `export_graphviz()` untuk membuat file `.dot`, yang kemudian dapat dikonversi menjadi gambar (misalnya PNG) menggunakan alat `dot` dari paket Graphviz.

**Membuat Prediksi**

Decision Tree membuat prediksi dengan menelusuri simpul dari root hingga node daun (leaf node). Setiap node internal mengajukan pertanyaan berdasarkan fitur, dan jalur yang diambil bergantung pada jawaban (True atau False). Node daun tidak memiliki anak dan memberikan prediksi kelas.

* **Contoh:** Jika panjang kelopak $\le$ 2.45 cm, Decision Tree memprediksi Iris setosa. Jika panjang kelopak $>$ 2.45 cm dan lebar kelopak $\le$ 1.75 cm, diprediksi Iris versicolor. Jika panjang kelopak $>$ 2.45 cm dan lebar kelopak $>$ 1.75 cm, diprediksi Iris virginica.

Decision Trees membutuhkan sedikit persiapan data dan tidak memerlukan penskalaan atau pemusatan fitur.

* **Atribut Node:**
    * `samples`: Menunjukkan berapa banyak instance pelatihan yang berlaku untuk node tersebut.
    * `value`: Menunjukkan berapa banyak instance pelatihan dari setiap kelas yang berlaku untuk node tersebut.
    * `gini`: Mengukur kemurnian node. Node dianggap "murni" ($gini=0$) jika semua instance pelatihan yang berlaku untuknya termasuk dalam kelas yang sama.

**Gini Impurity**

Gini impurity ($G_i$) dihitung dengan rumus:
$$G_i = 1 - \sum_{k=1}^{n} p_{i,k}^2$$
di mana $p_{i,k}$ adalah rasio instance kelas $k$ di node $i$.

Scikit-Learn menggunakan algoritma CART yang menghasilkan binary trees (setiap node non-daun memiliki dua anak).

**Interpretasi Model: White Box vs. Black Box**

Decision Trees disebut "white box models" karena keputusannya mudah diinterpretasikan dan dipahami. Sebaliknya, model seperti Random Forests atau neural networks adalah "black box models" karena sulit menjelaskan alasan di balik prediksinya secara sederhana.

**Estimasi Probabilitas Kelas**

Decision Tree juga dapat memperkirakan probabilitas suatu instance termasuk dalam kelas tertentu. Ini dilakukan dengan menelusuri pohon ke node daun untuk instance tersebut, lalu mengembalikan rasio instance pelatihan dari kelas $k$ di node tersebut.

Contoh kode untuk memprediksi probabilitas dan kelas:

```python
tree_clf.predict_proba([[5, 1.5]])
# Output: array([[0., 0.90740741, 0.09259259]])

tree_clf.predict([[5, 1.5]])
# Output: array([1])
```
Ini menunjukkan probabilitas untuk setiap kelas (setosa, versicolor, virginica) dan kelas yang diprediksi (1 untuk versicolor).

**Algoritma Pelatihan CART**

Algoritma CART (Classification and Regression Tree) melatih Decision Tree dengan membagi training set menjadi dua subset berdasarkan fitur $k$ tunggal dan ambang batas $t_k$. Algoritma ini mencari pasangan $(k, t_k)$ yang menghasilkan subset paling murni (dengan bobot ukurannya). Fungsi biaya yang diminimalkan untuk klasifikasi adalah:
$$J(k, t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}$$
di mana $G_{left/right}$ adalah impurity subset kiri/kanan dan $m_{left/right}$ adalah jumlah instance di subset kiri/kanan.

Proses pembagian ini dilakukan secara rekursif hingga mencapai kedalaman maksimum (`max_depth`) atau tidak dapat menemukan pembagian yang mengurangi impurity. CART adalah algoritma "greedy" karena mencari split optimal di setiap level, tetapi tidak menjamin solusi global optimal. Menemukan pohon optimal adalah masalah NP-Complete.

**Kompleksitas Komputasi**

* **Prediksi:** Membutuhkan waktu $O(\log_2(m))$ karena melintasi pohon dari root ke daun, di mana $m$ adalah jumlah instance. Prediksi sangat cepat bahkan dengan training set yang besar.
* **Pelatihan:** Algoritma pelatihan membandingkan semua fitur pada semua sampel di setiap node, menghasilkan kompleksitas $O(n \times m \log_2(m))$, di mana $n$ adalah jumlah fitur. Untuk training set kecil, `presort=True` dapat mempercepat pelatihan, tetapi memperlambat untuk set yang lebih besar.

**Gini Impurity atau Entropy?**

Secara default, Gini impurity digunakan, tetapi entropy juga dapat dipilih dengan mengatur hyperparameter `criterion` ke "entropy". Entropy mengukur ketidakteraturan, dan bernilai nol ketika suatu set hanya berisi instance dari satu kelas. Rumus entropy untuk node $i$ adalah:
$$H_i = -\sum_{k=1}^{n} p_{i,k} \log_2(p_{i,k})$$
Umumnya, tidak ada perbedaan besar antara Gini impurity dan entropy; keduanya mengarah pada pohon yang serupa. Gini impurity sedikit lebih cepat dihitung. Gini impurity cenderung mengisolasi kelas yang paling sering dalam cabangnya sendiri, sementara entropy cenderung menghasilkan pohon yang sedikit lebih seimbang.

**Hyperparameter Regularisasi**

Decision Trees adalah model nonparametrik, artinya struktur model tidak ditentukan sebelum pelatihan dan dapat sangat cocok dengan data pelatihan, sering kali menyebabkan overfitting. Untuk mencegah overfitting, perlu membatasi kebebasan Decision Tree selama pelatihan (regularisasi).

Hyperparameter regularisasi di Scikit-Learn meliputi:
* `max_depth`: Kedalaman maksimum Decision Tree (default: `None`, tak terbatas). Mengurangi `max_depth` akan meregularisasi model.
* `min_samples_split`: Jumlah minimum sampel yang harus dimiliki node sebelum dapat dibagi.
* `min_samples_leaf`: Jumlah minimum sampel yang harus dimiliki node daun.
* `min_weight_fraction_leaf`: Sama seperti `min_samples_leaf` tetapi sebagai pecahan dari total jumlah instance berbobot.
* `max_leaf_nodes`: Jumlah maksimum node daun.
* `max_features`: Jumlah maksimum fitur yang dievaluasi untuk pembagian di setiap node.

Meningkatkan hyperparameter `min_*` atau mengurangi hyperparameter `max_*` akan meregularisasi model.

Algoritma lain bekerja dengan melatih Decision Tree tanpa batasan, kemudian memangkas node yang tidak perlu. Node dianggap tidak perlu jika peningkatan kemurnian yang diberikannya tidak signifikan secara statistik, seringkali diuji dengan $\chi^2$ test dan p-value.

**Regresi**

Decision Trees juga dapat melakukan tugas regresi menggunakan kelas `DecisionTreeRegressor`. Perbedaan utamanya adalah bahwa setiap node memprediksi nilai, bukan kelas. Nilai prediksi untuk node daun adalah nilai target rata-rata dari semua instance pelatihan yang terkait dengan node tersebut.

Fungsi biaya CART untuk regresi adalah:
$$J(k,t_{k})=\frac{m_{left}}{m}MSE_{left}+\frac{m_{right}}{m}MSE_{right}$$
di mana $MSE_{node}=\sum_{i\in node}(\hat{y}_{node}-y^{(i)})^{2}$ dan $\hat{y}_{node}=\frac{1}{m_{node}}\sum_{i\in node}y^{(i)}$.

Sama seperti klasifikasi, Decision Trees rentan terhadap overfitting dalam tugas regresi. Regularisasi, seperti mengatur `min_samples_leaf`, sangat penting untuk model regresi yang lebih masuk akal.

**Ketidakstabilan**

Meskipun Decision Trees kuat, mereka memiliki beberapa batasan:
* **Orthogonal Decision Boundaries:** Decision Trees cenderung membuat batas keputusan ortogonal (tegak lurus terhadap sumbu), membuatnya sensitif terhadap rotasi training set. PCA dapat membantu mengatasi masalah ini.
* **Sensitivitas terhadap Variasi Data:** Decision Trees sangat sensitif terhadap variasi kecil dalam data pelatihan. Bahkan perubahan kecil atau pemilihan `random_state` yang berbeda dapat menghasilkan model yang sangat berbeda.

Random Forests dapat membatasi ketidakstabilan ini dengan merata-ratakan prediksi dari banyak pohon.